In [1]:
import os 
import glob
import sqlite3
import pandas as pd


In [2]:
try:
    os.remove('airline2.db')
# if something goes wrong, just let it go through
except OSError:
    pass

In [3]:
conn = sqlite3.connect('airline2.db')

In [4]:
os.getcwd()

'/Users/zilynnow/Desktop/ST2195_ProgrammingforDataScience/Assignment/Practice_Assignment_03/python_sql'

In [5]:
os.chdir("/Users/zilynnow/Desktop/ST2195_ProgrammingforDataScience/Assignment/Practice_Assignment_03/ontime")
extension = "csv"
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
ontime = pd.concat([pd.read_csv(f) for f in all_filenames])

/Users/zilynnow/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3357: DtypeWarning: Columns (22) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [6]:
print(ontime)

         Year  Month  DayofMonth  DayOfWeek  DepTime  CRSDepTime  ArrTime  \
0        2001      1          17          3   1806.0        1810   1931.0   
1        2001      1          18          4   1805.0        1810   1938.0   
2        2001      1          19          5   1821.0        1810   1957.0   
3        2001      1          20          6   1807.0        1810   1944.0   
4        2001      1          21          7   1810.0        1810   1954.0   
...       ...    ...         ...        ...      ...         ...      ...   
7140591  2005     12          22          4   1652.0        1655   1815.0   
7140592  2005     12          22          4   1825.0        1825   1858.0   
7140593  2005     12          22          4   1507.0        1511   1641.0   
7140594  2005     12          22          4    924.0         925   1056.0   
7140595  2005     12          22          4   1345.0        1344   1621.0   

         CRSArrTime UniqueCarrier  FlightNum  ... TaxiIn  TaxiOut  Cancelle

In [7]:
airports = pd.read_csv("/Users/zilynnow/Desktop/ST2195_ProgrammingforDataScience/Assignment/Practice_Assignment_03/dataverse_files/airports.csv")
carriers = pd.read_csv("/Users/zilynnow/Desktop/ST2195_ProgrammingforDataScience/Assignment/Practice_Assignment_03/dataverse_files/carriers.csv")
planes = pd.read_csv("/Users/zilynnow/Desktop/ST2195_ProgrammingforDataScience/Assignment/Practice_Assignment_03/dataverse_files/plane-data.csv")

In [8]:
ontime.to_sql("ontime", con = conn, index = False) 

In [9]:
airports.to_sql("airports", con = conn, index = False)
carriers.to_sql("carriers", con = conn, index = False)
planes.to_sql("planes", con = conn, index = False)

In [10]:
c = conn.cursor()

In [11]:
c.execute('''
SELECT name 
FROM sqlite_master 
WHERE type='table'
''').fetchall()

[('ontime',), ('airports',), ('carriers',), ('planes',)]

Q1: Which plane model has the lowest associated average departure delay (excluding cancelled and diverted flights)?

In [15]:
q1= c.execute('''
SELECT model AS model, AVG(ontime.DepDelay) AS avg_DepDelay
FROM ontime INNER JOIN planes USING (tailnum)
WHERE ontime.DepDelay>0 & ontime.Diverted == 0 & ontime.Cancelled == 0
GROUP BY model
ORDER BY avg_DepDelay ASC
''').fetchall()

pd.DataFrame(q1).to_sql("q1", con = conn, index = False)

In [16]:
print(q1[0][0],"has the lowest associated average departure delay.")

737-2Y5 has the lowest associated average departure delay.


In [17]:
q1 = c.execute('''
ALTER TABLE q1
RENAME COLUMN "0" TO model 
''').fetchall()

q1 = c.execute('''
ALTER TABLE q1
RENAME COLUMN "1" TO avg_DepDelay
''').fetchall()

Q2: Which city has the highest number of inbound flights (excluding cancelled flights?)

In [18]:
q2= c.execute('''
SELECT airports.city AS airport, COUNT(*) as total
FROM airports INNER JOIN ontime
ON ontime.Dest = airports.iata
WHERE ontime.Cancelled = 0 
GROUP BY airports.city
ORDER BY total DESC 
''').fetchall()

pd.DataFrame(q2).to_sql("q2", con = conn, index = False)

In [19]:
print(q2[0][0],"highest number of inbound flights (excluding cancelled flights).")

Chicago highest number of inbound flights (excluding cancelled flights).


In [20]:
q2 = c.execute('''
ALTER TABLE q2
RENAME COLUMN "0" TO airport 
''').fetchall()

q2 = c.execute('''
ALTER TABLE q2
RENAME COLUMN "1" TO total
''').fetchall()

Q3: Which carrier has the highest number of cancelled flights?

In [30]:
q3 = c.execute('''
SELECT carriers.Description AS carrier, COUNT(*) as total
FROM carriers INNER JOIN ontime 
ON ontime.UniqueCarrier = carriers.Code
WHERE ontime.Cancelled = 1
GROUP BY carriers.Description
ORDER BY total DESC 
''').fetchall()
pd.DataFrame(q3).to_sql("q3", con = conn, index = False)

In [31]:
print(q3[0][0],"has the highest number of cancelled flights.")

Delta Air Lines Inc. has the highest number of cancelled flights.


In [32]:
q3 = c.execute('''
ALTER TABLE q3
RENAME COLUMN "0" TO carrier
''').fetchall()

q3 = c.execute('''
ALTER TABLE q3
RENAME COLUMN "1" TO total
''').fetchall()

Q4: Which carrier has the highest number of cancelled flights, relative to their number of total flights?

In [24]:
UniqueCarrierCount = c.execute('''
SELECT carriers.Description AS carrier, COUNT(*) as denominator
FROM carriers INNER JOIN ontime 
ON ontime.UniqueCarrier = carriers.Code
GROUP BY carriers.Description
ORDER BY denominator DESC
''').fetchall()
pd.DataFrame(UniqueCarrierCount).to_sql("UniqueCarrierCount", con = conn, index = False)


In [25]:
UniqueCarrierCount = c.execute('''
ALTER TABLE UniqueCarrierCount
RENAME COLUMN "0" TO carrier
''').fetchall()

UniqueCarrierCount = c.execute('''
ALTER TABLE UniqueCarrierCount
RENAME COLUMN "1" TO denominator
''').fetchall()

In [26]:
q4 = c.execute('''
SELECT carrier AS carrier, total* 1./denominator as ratio
FROM q3 INNER JOIN UniqueCarrierCount USING(carrier)
ORDER BY ratio DESC
''').fetchall()

pd.DataFrame(q4).to_sql("q4", con = conn, index = False)

In [27]:
print(q4[0][0],"has the highest number of cancelled flights, relative to their number of total flights.")

American Eagle Airlines Inc. has the highest number of cancelled flights, relative to their number of total flights.


In [28]:
q4 = c.execute('''
ALTER TABLE q4
RENAME COLUMN "0" TO carrier
''').fetchall()

q4 = c.execute('''
ALTER TABLE q4
RENAME COLUMN "1" TO ratio
''').fetchall()

In [33]:
conn.commit()

In [34]:
conn.close()